In [246]:
import numpy as np

f = open("data.txt", "r")
data = f.read()
data = data.split('\n\n')
data = [x.split('\n') for x in data]



In [247]:
def parseData(data):
    cleaned = []
    for d in data:
        title, arr = d[0],d[1:]
        title = title.split(' ')[1].strip(':')
        arr = np.array([list(x) for x in arr])
        cleaned.append((title, arr))
    return cleaned
cleanedData = parseData(data)

In [248]:
# Part 1 Jurassic Jigsaw
# Reconstruct the image of 12x12 grid, each tile shares a side the the adjacent array
# side tiles have 1 unique edge, corner tiles have 2 unique edges
# so for part 1 we just need to find 4 tiles that have 2 unique edges
# tiles can be rotated and flipped to a random orientation
# rotating doesnt change the order of the edge?? just flipping vertically and horizontally
# create a dict where each edge is a key and store the tile title in a list


def pairEdges(data):
    edges = {}
    for title, arr in data:
        # print(title)
        eachEdge = []
        eachEdge.append(arr[0,:]) # top
        eachEdge.append(arr[-1,:]) # bottom
        eachEdge.append(arr[:,0]) # left
        eachEdge.append(arr[:,-1]) # right
        # print(eachEdge)
        for e in eachEdge:
            lookup = ''.join(e)
            # if i == 1:
            #     lookup = lookup[::-1]
            if lookup in edges:
                edges[lookup].append(title)
            elif lookup[::-1] in edges:
                edges[lookup[::-1]].append(title)
            else:
                edges[lookup] = [title]
    return edges


def getUniqueEdges(edgePairs):
    """
    uniqueEdges: Dict of how many times a tile has a unique edge
    uniqueEdgesVal: Dict of the unique edges for each tile
    """

    uniqueEdges = {}
    uniqueEdgesVal = {}
    for p in edgePairs:
        if len(edgePairs[p]) == 1:
            title = edgePairs[p][0]
            if title in uniqueEdges:
               uniqueEdges[title] = uniqueEdges[title] + 1
               uniqueEdgesVal[title].append(p)
            else:
                uniqueEdgesVal[title] = [p]
                uniqueEdges[title] = 1
    return uniqueEdges, uniqueEdgesVal
def getCornerTiles(uniqueEdges):
    cornerTiles = []
    for u in uniqueEdges:
        if uniqueEdges[u] == 2:
            cornerTiles.append(u)
    return cornerTiles
    
    
edgePairs = pairEdges(cleanedData)
uniqueEdges, uniqueEdgesVal = getUniqueEdges(edgePairs)
cornerTiles = getCornerTiles(uniqueEdges)
tiles = dict(cleanedData)
cumPro = 1
for a in cornerTiles:
    cumPro *= int(a)
print(cumPro)


15670959891893


In [249]:
# Part 2 
# there's some ~ s p o o k y ~ sea monsters out and about
# first we gotta construct the image
# remove all the common edges from the tiles 
# then count how many times this lad appears
                  # 
#    ##    ##    ###
 #  #  #  #  #  #   

In [250]:
def getCornerAndSideTiles(uniqueEdges):
    sideTiles = []
    cornerTiles = []
    for u in uniqueEdges:
        if uniqueEdges[u] == 1:
            sideTiles.append(u)
        if uniqueEdges[u] == 2:
            cornerTiles.append(u)
    return sideTiles, cornerTiles

In [251]:
def getTilesByLocation(tiles, uniqueEdges):
    sideTiles, cornerTiles = getCornerAndSideTiles(uniqueEdges)
    innerTiles = []
    for t in tiles: 
        if t not in sideTiles and t not in cornerTiles:
            innerTiles.append(t)
    assert(len(tiles) == 144)
    assert(len(cornerTiles) == 4)
    assert(len(sideTiles) == 144-100-4)
    assert(len(innerTiles) == 100)
    return cornerTiles, sideTiles, innerTiles

In [252]:
def getCornerOrientation(tileTitle,tiles,uniqueEdgesVal):
    topStr = ''.join(tiles[tileTitle][0,:])
    bottomStr = ''.join(tiles[tileTitle][-1,:])
    leftStr = ''.join(tiles[tileTitle][:,0])
    rightStr = ''.join(tiles[tileTitle][:,-1])

    top = None # true if top, false if bottom
    topReversed = None # if needs to be vertically flipped
    side = None # true if left, false if right
    sideReversed = None #if needs to be horizontally flipped

    if topStr in uniqueEdgesVal[tileTitle]:
        top = True
        topReversed = False
    elif topStr[::-1] in uniqueEdgesVal[tileTitle]:
        top = True
        topReversed = True
    elif bottomStr in uniqueEdgesVal[tileTitle]:
        top = False
        topReversed = False
    elif bottomStr[::-1] in uniqueEdgesVal[tileTitle]:
        top = False
        topReversed = True

    if leftStr in uniqueEdgesVal[tileTitle]:
        side = True
        sideReversed = False
    elif leftStr[::-1] in uniqueEdgesVal[tileTitle]:
        side = True
        sideReversed = True
    elif rightStr in uniqueEdgesVal[tileTitle]:
        side = False
        sideReversed = False
    elif rightStr[::-1] in uniqueEdgesVal[tileTitle]:
        side = False
        sideReversed = True

    return top, topReversed, side, sideReversed


In [253]:
"""
(0,0) up, left
(0,11) up, right
(11,0) down, left
(11,11) down, right
"""

def getOutSideFacingCorner(tileTitle,tiles,uniqueEdgesVal, i,j):

    top, topReversed, side, sideReversed = getCornerOrientation(tileTitle,tiles,uniqueEdgesVal)
    # print(top, side)
    # print('top',tiles[tileTitle][0,:])
    # (0,0) up, left
    # (0,11) up, right
    # (11,0) down, left
    # (11,11) down, right

    if (i,j) == (0,0):
        # print('top left')
        while top != True or side != True:
            tiles[tileTitle] = np.rot90(tiles[tileTitle])
            top, topReversed, side, sideReversed = getCornerOrientation(tileTitle,tiles,uniqueEdgesVal)
            # print(tiles[tileTitle])

    # find the current orientation of the unique edges
    # rotate the the tile until it fits and determine if it need to be flipped


In [254]:
# image is 12x12, corner tiles have 2 unique edges, side tiles have 1
# doesnt matter what order we generate the image in since we can just flip it
# start with a corner tile with the two unique edges facing outward then keep adding on side tiles
# we're just gonna rotate and flip the tiles in the tiles dict to keep track of everything
def renderImage(tiles, edgePairs, uniqueEdges, uniqueEdgesVal):
    cornerTiles, sideTiles, innerTiles = getTilesByLocation(tiles, uniqueEdges)
    image = [['' for i in range(12)] for j in range(12)] 
    count = 0
    for i in range(12):
        for j in range(12):
            # print(i,j)
            corner = (i,j) == (0,0)
            side = (i == 0 or i == 11 or j == 0 or j == 11) and not corner
            if corner:
                image[i][j] = cornerTiles.pop()
                getOutSideFacingCorner(image[i][j],tiles,uniqueEdgesVal, i,j)
                # print('curr')
                # print(tiles[image[i][j]])
            elif side:
                # if on top or bottom we can just look at the edgePair on the last tile on the left
                # if on left or right we can just look at the edgePair on the last tile on the top
                if i == 0 or (i == 11 and j !=0):
                    # get right hand side of tile on the left
                    # print(''.join(tiles[image[i][j-1]][:,-1]))
                    lookup = ''.join(tiles[image[i][j-1]][:,-1])
                    p = -1
                    reversedSide = False

                    # look up the last edge and get the title of the current tile
                    if lookup in edgePairs:
                        p = edgePairs[lookup].index(image[i][j-1])
                        currTitle = edgePairs[lookup][1-p]
                    elif lookup[::-1] in edgePairs:
                        p = edgePairs[lookup[::-1]].index(image[i][j-1])
                        currTitle = edgePairs[lookup[::-1]][1-p]


                    leftStr = ''.join(tiles[currTitle][:,0])
                   
                    while leftStr != lookup and leftStr != lookup[::-1]:
                        tiles[currTitle] = np.rot90(tiles[currTitle])
                        leftStr = ''.join(tiles[currTitle][:,0])
                        
                    if leftStr != lookup:
                        tiles[currTitle] = np.flipud(tiles[currTitle])

                    image[i][j] = currTitle
                    # print('curr')
                    # print(tiles[image[i][j]])
                else:
                    lookup = ''.join(tiles[image[i-1][j]][-1,:])
                    p = -1
                    reversedSide = False

                    # look up the last edge and get the title of the current tile
                    if lookup in edgePairs:
                        p = edgePairs[lookup].index(image[i-1][j])
                        currTitle = edgePairs[lookup][1-p]
                    elif lookup[::-1] in edgePairs:
                        p = edgePairs[lookup[::-1]].index(image[i-1][j])
                        currTitle = edgePairs[lookup[::-1]][1-p]


                    topStr = ''.join(tiles[currTitle][0,:])
                   
                    while topStr != lookup and topStr != lookup[::-1]:
                        tiles[currTitle] = np.rot90(tiles[currTitle])
                        topStr = ''.join(tiles[currTitle][0,:])
                        
                    if topStr != lookup:
                        tiles[currTitle] = np.fliplr(tiles[currTitle])

                    image[i][j] = currTitle
                    # print('prev')
                    # print(tiles[image[i-1][j]])
                    # print('curr')
                    # print(tiles[image[i][j]])

                
            else:
                lookup = ''.join(tiles[image[i][j-1]][:,-1])
                p = -1
                reversedSide = False
                # look up the last edge and get the title of the current tile
                if lookup in edgePairs:
                    p = edgePairs[lookup].index(image[i][j-1])
                    currTitle = edgePairs[lookup][1-p]
                elif lookup[::-1] in edgePairs:
                    p = edgePairs[lookup[::-1]].index(image[i][j-1])
                    currTitle = edgePairs[lookup[::-1]][1-p]
                leftStr = ''.join(tiles[currTitle][:,0])
                
                while leftStr != lookup and leftStr != lookup[::-1]:
                    tiles[currTitle] = np.rot90(tiles[currTitle])
                    leftStr = ''.join(tiles[currTitle][:,0])
                    
                if leftStr != lookup:
                    tiles[currTitle] = np.flipud(tiles[currTitle])
                image[i][j] = currTitle
                # print('curr')
                # print(tiles[image[i][j]])
        #     break
    return image

    # get list of all corner tiles, side tiles, and inner tiles
image = renderImage(tiles, edgePairs, uniqueEdges, uniqueEdgesVal)

SyntaxError: invalid syntax (<ipython-input-11-f244a3bc6600>, line 11)

In [255]:
def megaArray(image, tiles):
    # for each row of the image 
    # print()
    print(np.array(image).shape)
    arr = []
    for i in range(12):
        print("s",image[i])
        for z in image[i]:
            # print(tiles[z])

            tiles[z] = tiles[z][1:-1,1:-1]
            # print(tiles[z])

        arr.append(np.concatenate([tiles[z] for z in image[i]],  axis=1))

        # for x in 
        #     print(x)
        #     arr.append(np.concatenate([tiles[z] for z in x],  axis=1))
    fin = np.concatenate(arr, axis=0)
    print(fin.shape)
    # fin = np.flipud(fin)
    # fin = np.fliplr(fin)

    # fin = np.rot90(fin)
    # fin = np.rot90(fin)
    # fin = np.rot90(fin)


    np.savetxt('test1.txt', fin, fmt='%s', delimiter='')
    return fin
fin = megaArray(image, tiles)


(12, 12)
s ['1151', '2341', '2311', '1873', '2269', '2731', '3181', '2531', '1487', '1291', '3167', '3079']
s ['1303', '3187', '3943', '3229', '3209', '2909', '1879', '3251', '1759', '3067', '3803', '1019']
s ['2111', '2741', '3259', '3217', '2791', '3169', '3739', '1171', '3203', '2711', '1627', '2647']
s ['2617', '1301', '3001', '2887', '2551', '1571', '1511', '1723', '3299', '2221', '3823', '2819']
s ['3119', '1451', '3089', '2131', '2383', '1637', '2423', '3121', '2153', '1721', '1153', '2459']
s ['1283', '2539', '1361', '2549', '2083', '3011', '1933', '3083', '1877', '2447', '2467', '1997']
s ['2137', '2657', '1481', '2957', '1117', '2971', '2503', '1733', '2543', '3797', '1039', '3461']
s ['2293', '1259', '3049', '2963', '1783', '3191', '2833', '2011', '3571', '3989', '2371', '1049']
s ['3727', '1061', '1409', '3911', '2161', '1601', '2333', '3767', '2063', '1009', '1801', '2339']
s ['3637', '3533', '3539', '3257', '3947', '3691', '2027', '3329', '2851', '1847', '1543', '2473']
s

In [256]:
# def findMonsters(fin):
    # print(fin)
    # pattern = """\
    #                   #
    # #....##....##....###
    # #  #  #  #  #  #    """.splitlines()
    # # print(pattern)
    # for i in range(len(fin)-1):
        # print(fin[i
            # print(j)
            # 
            # if fin[i][j] == '#' and fin[i+1][j] == "#" and fin[i+1][j-1] == "#" and fin[i+1][j+1] == "#":
            #     print(i,j)

# findMonsters(fin)

In [259]:
total = 2534
monsters = 38
# there like 40 possible monsters using regex to search for "#....##....##....###"
total - 15 * monsters

1964

In [258]:
# after some guessing I know its lower than 40 and higher than 30
# after guessing the answer was 38
# this was trash 